In [2]:
import numpy as np
import pandas as pd
import scipy
from scipy import io
import statsmodels.api as sm
import time
from numpy import diag, sqrt, log, trace
from numpy.linalg import inv

## CWMR

Confidence Weighted Mean Reversion Strategy for Online Portfolio Selection. ACM Transactions on Knowledge Discovery from Data 7, 1 (2013), 4:1–4:38.

Reference:
B. Li, S. C. H. Hoi, P. Zhao, and V. Gopalkrishnan.
https://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.890.4606&rep=rep1&type=pdf

In [19]:
data=pd.read_csv('./Data_OLPS/nyse-o.csv',encoding="gb18030")
# data=pd.read_csv('./data/msci.csv',encoding="gb18030")

print('data.shape：',data.shape) 

#data.info()
N=data.shape[0]
d=data.shape[1]
x=np.zeros((N,d))
x=data.to_numpy()
x

data.shape： (5651, 36)


array([[1.01515, 1.02765, 1.04183, ..., 1.00578, 0.99697, 0.99752],
       [1.01493, 1.04036, 0.98905, ..., 1.00958, 0.99088, 1.00248],
       [1.     , 0.97629, 0.97786, ..., 1.     , 1.02761, 0.99752],
       ...,
       [0.99029, 0.9966 , 0.99605, ..., 0.99216, 1.00461, 0.99273],
       [0.99265, 1.00683, 1.     , ..., 0.99209, 1.02752, 1.00366],
       [0.99753, 1.00339, 1.01984, ..., 1.01195, 1.     , 0.99635]])

In [20]:
def simplex_proj(y):
    """ Projection of y onto simplex. """
    m = len(y)
    bget = False

    s = sorted(y, reverse=True)
    tmpsum = 0.

    for ii in range(m-1):
        tmpsum = tmpsum + s[ii]
        tmax = (tmpsum - 1) / (ii + 1);
        if tmax >= s[ii+1]:
            bget = True
            break

    if not bget:
        tmax = (tmpsum + s[m-1] -1)/m

    return np.maximum(y-tmax,0.)


def CWMR_Var(x, x_bar, mu, sigma, M, V, theta, eps):
    # lambda from equation 7
    a = 2 * phi * V**2 - 2 * phi * x_bar * V * W
    b = 2 * phi * eps * V - 2 * phi * V * M + V - x_bar * W
    c = eps - M - phi * V
    t1 = b
    t2 = sqrt(b**2 - 4 * a * c)
    t3 = 2 * a
    if((a!=0)&(np.isreal(t2))&(t2>0)):
        gamma1 = (-t1+t2)/t3 
        gamma2 = (-t1-t2)/t3
        lambd = np.maximum(np.maximum(gamma1,gamma2),0)
    elif(a == 0) & (b != 0):
        gamma3 = -c/b
        lambd = np.maximum(gamma3,0)   
    else:
        lambd = 0
    mu = mu - lambd * np.dot(sigma,(x - x_bar))/ M
    sigma = inv(inv(sigma) + 2 * lambd * phi * diag(x)**2)
    return mu, sigma

def CWMR_Stdev(x, x_bar, mu, sigma, M, V, theta, eps):
    # lambda from equation 7
    a = (V - x_bar * W + phi**2 * V / 2)**2 - (phi**4) * (V**2) / 4
    b = 2 * (eps - M) * (V - x_bar * W + (phi**2) * V / 2)
    c = (eps - M)**2 - (phi**2) * V
    t1 = b
    t2 = sqrt(b**2 - 4 * a * c)
    t3 = 2 * a
    if((a!=0)&(np.isreal(t2))&(t2>0)):
        gamma1 = (-t1+t2)/t3 
        gamma2 = (-t1-t2)/t3
        lambd = np.maximum(np.maximum(gamma1,gamma2),0)
    elif(a == 0) & (b != 0):
        gamma3 = -c/b
        lambd = np.maximum(gamma3,0)   
    else:
        lambd = 0
    mu = mu - lambd * np.dot(sigma,(x - x_bar))/ M
    sigma = inv(inv(sigma) + 2 * lambd * phi * diag(x)**2)
    return mu, sigma

In [21]:
start = time.time()
# INPUT:
phi=0.95 # Confidence parameter for profitable mean reversion portfolio.Recommended value is 0.95.
eps=-0.5 # Mean reversion threshold (expected return on current day must be lower than this threshold). Recommended value is -0.5.
# INITIALIZE:
cum_ret=1
lambd = 0
mu = np.ones(d)/d # mu: last portfolio mean
sigma = np.eye(d)/(d**2) # sigma: last diagonal covariance matrix
weight=mu/np.sum(mu)
theta = scipy.stats.norm.ppf(phi) # Quantile function (inverse of CDF
daily_r=np.ones(N)

for t in range(N-1):
    # 4. Calculate the following variables    
    M = np.dot(mu.T,x[t])
    V = np.dot(np.dot(x[t].T, sigma),x[t])
    W = np.dot(np.dot(x[t].T, sigma),np.ones(d))
    x_bar = (np.dot(np.ones(d),np.dot(sigma,x[t]))/np.dot((np.dot(np.ones(d),sigma)),np.ones(d)))
    # 5. Update mu and sigma
#     mu, sigma = CWMR_Var(x[t], x_bar, mu, sigma, M, V, theta, eps)
    mu, sigma = CWMR_Stdev(x[t], x_bar, mu, sigma, M, V, theta, eps)
    # 6. Normalize mu and sigma    
    mu = simplex_proj(mu)
    sigma = sigma / (d**2 * trace(sigma))
    weight = mu
#     tc = gamma/2 * (np.abs(((x[t+1]*weight)/ (np.dot(x[t+1],weight)))[1:] - weight[1:])).sum()
    daily_r[t] = np.dot(x[t+1],weight) #* (1-tc)
    cum_ret = cum_ret * daily_r[t] 
print("Cumulative return:",cum_ret,"-----> Expressed by scientific counting:",f'{cum_ret:1.2e}') 

end = time.time()
print("Program running time:",end-start)

Cumulative return: 1.8083607416553684e+16 -----> Expressed by scientific counting: 1.81e+16
Program running time: 1.978968858718872


In [22]:
daily_r = pd.DataFrame(daily_r,index = data.index,columns=['daily return'])
# print(daily_r)
mean_daily = daily_r['daily return'].mean()
print("Average daily rate of return:",mean_daily)
mean_return_annualized = mean_daily**252 - 1
# print("mean_return_annualized:",mean_return_annualized)

std_daily = np.std(np.array(daily_r))
variance_daily = std_daily ** 2
std_annualized = std_daily*np.sqrt(252)
variance_annualized = std_annualized ** 2
volatility = std_daily * np.sqrt(252)
print("Volatility is: ",volatility)
SR = mean_daily/std_daily
print("Daily Sharpe ratio is: ",SR)
ASR=np.sqrt(252)*SR
maximum = np.maximum.accumulate(np.array(daily_r))
MDD = ((maximum - np.array(daily_r)) / maximum).max()
print("Max Drawdown is: ",MDD)
CS = mean_return_annualized/MDD
print("Calmar Ratio is: ",CS)

Average daily rate of return: 1.007216867773141
Volatility is:  0.5457093601276874
Daily Sharpe ratio is:  29.299611219572554
Max Drawdown is:  0.387542627741314
Calmar Ratio is:  13.220256346787709
